In [1]:
%run "../PyUtils/common.ipynb"

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [2]:
%%html
<style>
#holder { border: 10px dashed #ccc; width: 300px; min-height: 300px; margin: 20px auto;}
#holder.hover { border: 10px dashed #0c0; }
#holder img { display: block; margin: 10px auto; }
#holder p { margin: 10px; font-size: 14px; }
progress { width: 100%; }
progress:after { content: '%'; }
.fail { background: #c00; padding: 2px; color: #fff; }
.hidden { display: none !important;}
</style>

<article>
   <form enctype="multipart/form-data" action="cgi-bin/savefile.py" method="post">
   File: <input type="file" name="filename" /> <input type="submit" value="Upload" />
   </form>
<br/>
  <div id="holder">
  </div>
  <p id="upload" class="hidden">
      <label>Drag & drop not supported, but you can still upload via this input field:<br>
      <input type="file"></label>
  </p>
  <p id="filereader">File API & FileReader API not supported</p>
  <p id="formdata">XHR2's FormData is not supported</p>
  <p id="progress">XHR2's upload progress isn't supported</p>
  <p>Upload progress: <progress id="uploadprogress" min="0" max="100" value="0">0</progress></p>
</article>

 <script>
var holder = document.getElementById('holder'),
    tests = {
      filereader: typeof FileReader != 'undefined',
      dnd: 'draggable' in document.createElement('span'),
      formdata: !!window.FormData,
      progress: "upload" in new XMLHttpRequest
    },
    support = {
      filereader: document.getElementById('filereader'),
      formdata: document.getElementById('formdata'),
      progress: document.getElementById('progress')
    },
    acceptedTypes = {
      'image/png': true,
      'image/jpeg': true,
      'image/gif': true,

    },
    progress = document.getElementById('uploadprogress'),
    fileupload = document.getElementById('upload');

"filereader formdata progress".split(' ').forEach(function (api) {
  if (tests[api] === false) {
    support[api].className = 'fail';
  } else {
    // FFS. I could have done el.hidden = true, but IE doesn't support
    // hidden, so I tried to create a polyfill that would extend the
    // Element.prototype, but then IE10 doesn't even give me access
    // to the Element object. Brilliant.
    support[api].className = 'hidden';
  }
});

function previewfile(file) {
  if (tests.filereader === true && acceptedTypes[file.type] === true) {
    var reader = new FileReader();
    reader.onload = function (event) {
      var image = new Image();
      image.src = event.target.result;
      image.width = 250; // a fake resize
      holder.appendChild(image);
    };

    reader.readAsDataURL(file);
  }  else {
    holder.innerHTML += '<p>Uploaded ' + file.name + ' ' + (file.size ? (file.size/1024|0) + 'K' : '');
    console.log(file);
  }
}

TestformData = null;
function postToServer(formData){
    // now post a new XHR request
    if (formData) {
      TestformData = formData;
      var xhr = new XMLHttpRequest();
      xhr.open('POST', 'cgi-bin/getfile.py');
      xhr.onload = function() {
      };
      xhr.send(formData);
    }
}

function readfiles(files) {
    debugger;
    var formData = tests.formdata ? new FormData() : null;
    for (var i = 0; i < files.length; i++) {
      if (tests.formdata) {
          formData.append('file', files[i]);
      }
      previewfile(files[i]);
    }
    if ( tests.formdata) {
        postToServer(formData)
    }
}
if (tests.dnd) {
  holder.ondragover = function () { this.className = 'hover'; return false; };
  holder.ondragend = function () { this.className = ''; return false; };
  holder.ondrop = function (e) {
    this.className = '';
    e.preventDefault();
    readfiles(e.dataTransfer.files);
  }
} else {
    //fileupload.className = 'hidden';
    fileupload.querySelector('input').onchange = function () {
        readfiles(this.files);
    };
}
</script>


In [ ]:
#Test.py
#!/usr/local/bin/python

print ('''Content-type:text/html\r\n\r\n
<html>
<head>
<title>Hello Word - First CGI Program</title>
</head>
<body>
<h2>Hello Word! This is my first CGI program</h2>
</body>
</html>
''')

#savefile.py
!/usr/local/bin/python

import cgi, os
import cgitb; cgitb.enable()

form = cgi.FieldStorage()

# Get filename here.
fileitem = form['filename']

# Test if the file was uploaded
if fileitem.filename:
   # strip leading path from file name to avoid
   # directory traversal attacks
   fn = os.path.basename(fileitem.filename)
   open('tmp/' + fn, 'wb').write(fileitem.file.read())

   message = 'The file "' + fn + '" was uploaded successfully'

else:
   message = 'No file was uploaded'

print ("""\
Content-Type: text/html\n
<html>
<body>
   <p>%s</p>
</body>
</html>
""" % (message,))

#getfile.py
#!/usr/local/bin/python

import cgi, cgitb
import tempfile
import datetime

suf=datetime.datetime.now().strftime("%Y%m%d-%H:%M:%S.jpeg")

form = cgi.FieldStorage() 
o = ""
with (open ("tmp/log.file", "a+b") ) as fb:
    fb.write(b'\n---------+++++++\n')
    for i,j in enumerate(form):
        c = "{} {} {}  ".format(i, j, j) #(val[:100]) ) ;
        o += c;
        fb.write(bytearray(c, 'utf8'));
        fb.write(b'\n-------------------')
        
        fp = tempfile.NamedTemporaryFile(delete=False, dir="tmp/",suffix=suf)
        o += str(fp.name)
        fp.write(bytes(form.getvalue(j) ));
        fp.flush()
        fp.close()
    


print ('''Content-type:text/html\r\n\r\n''' + o)

Content-type:text/html


<html>
<head>
<title>Hello Word - First CGI Program</title>
</head>
<body>
<h2>Hello Word! This is my first CGI program</h2>
</body>
</html>

Python 3.5.2 |Anaconda 4.2.0 (x86_64)| (default, Jul  2 2016, 17:52:12) 
[GCC 4.2.1 Compatible Apple LLVM 4.2 (clang-425.0.28)] on darwin
Type "help", "copyright", "credits" or "license" for more information.
>>> 